<a href="https://colab.research.google.com/github/ChoudharyImran/Lab-5-Exploring-the-Dataset/blob/main/M2DataWrangling_lab_v2_v1_COMPLETED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [3]:
!pip install pandas
!pip install matplotlib

## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [2]:
# Import necessary libraries
import pandas as pd

# Load the Stack Overflow survey data
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

# Display the first few rows
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [4]:
# Dataset overview
print("Dataset shape:", df.shape)
print("\nColumn data types and missing values:")
print(df.info())
print("\nMissing values count:")
print(df.isnull().sum().sort_values(ascending=False))

Dataset shape: (65437, 114)

Column data types and missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Columns: 114 entries, ResponseId to JobSat
dtypes: float64(13), int64(1), object(100)
memory usage: 56.9+ MB
None

Missing values count:
AINextMuch less integrated    64289
AINextLess integrated         63082
AINextNo change               52939
AINextMuch more integrated    51999
EmbeddedAdmired               48704
                              ...  
MainBranch                        0
Age                               0
Employment                        0
Check                             0
ResponseId                        0
Length: 114, dtype: int64


<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [5]:
# Generate basic statistics for numerical columns
print("Basic statistics for numerical columns:")
print(df.describe())
print("\nNumerical columns:")
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(numerical_cols)

Basic statistics for numerical columns:
         ResponseId      CompTotal       WorkExp  JobSatPoints_1  \
count  65437.000000   3.374000e+04  29658.000000    29324.000000   
mean   32719.000000  2.963841e+145     11.466957       18.581094   
std    18890.179119  5.444117e+147      9.168709       25.966221   
min        1.000000   0.000000e+00      0.000000        0.000000   
25%    16360.000000   6.000000e+04      4.000000        0.000000   
50%    32719.000000   1.100000e+05      9.000000       10.000000   
75%    49078.000000   2.500000e+05     16.000000       22.000000   
max    65437.000000  1.000000e+150     50.000000      100.000000   

       JobSatPoints_4  JobSatPoints_5  JobSatPoints_6  JobSatPoints_7  \
count    29393.000000    29411.000000    29450.000000     29448.00000   
mean         7.522140       10.060857       24.343232        22.96522   
std         18.422661       21.833836       27.089360        27.01774   
min          0.000000        0.000000        0.000000  

### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [6]:
# Identify inconsistent entries in Country column
print("Unique values in Country column (first 20):")
print(df['Country'].value_counts().head(20))
print("\nTotal unique countries:", df['Country'].nunique())

# Check for potential inconsistencies
print("\nPotential inconsistencies (countries with very low counts):")
country_counts = df['Country'].value_counts()
low_count_countries = country_counts[country_counts <= 2]
print(low_count_countries)

Unique values in Country column (first 20):
Country
United States of America                                11095
Germany                                                  4947
India                                                    4231
United Kingdom of Great Britain and Northern Ireland     3224
Ukraine                                                  2672
France                                                   2110
Canada                                                   2104
Poland                                                   1534
Netherlands                                              1449
Brazil                                                   1375
Italy                                                    1341
Australia                                                1260
Spain                                                    1123
Sweden                                                   1016
Russian Federation                                        925
Switzerland       

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [7]:
# Standardize Country entries
# Map common variations to standard names
country_mapping = {
    'United States of America': 'United States',
    'USA': 'United States',
    'US': 'United States',
    'UK': 'United Kingdom',
    'Great Britain': 'United Kingdom',
    'England': 'United Kingdom'
}

# Apply mapping
df['Country'] = df['Country'].replace(country_mapping)

print("After standardization - Top 10 countries:")
print(df['Country'].value_counts().head(10))

# Standardize EdLevel if it exists
if 'EdLevel' in df.columns:
    print("\nEducation levels before standardization:")
    print(df['EdLevel'].value_counts())

    # Clean up education level entries
    df['EdLevel'] = df['EdLevel'].str.strip()
    df['EdLevel'] = df['EdLevel'].str.title()

    print("\nEducation levels after standardization:")
    print(df['EdLevel'].value_counts())

After standardization - Top 10 countries:
Country
United States                                           11095
Germany                                                  4947
India                                                    4231
United Kingdom of Great Britain and Northern Ireland     3224
Ukraine                                                  2672
France                                                   2110
Canada                                                   2104
Poland                                                   1534
Netherlands                                              1449
Brazil                                                   1375
Name: count, dtype: int64

Education levels before standardization:
EdLevel
Bachelor’s degree (B.A., B.S., B.Eng., etc.)                                          24942
Master’s degree (M.A., M.S., M.Eng., MBA, etc.)                                       15557
Some college/university study without earning a degree                

### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [8]:
# One-hot encode the Employment column
if 'Employment' in df.columns:
    print("Original Employment values:")
    print(df['Employment'].value_counts())

    # Perform one-hot encoding
    employment_encoded = pd.get_dummies(df['Employment'], prefix='Employment')

    # Add encoded columns to dataframe
    df = pd.concat([df, employment_encoded], axis=1)

    print("\nOne-hot encoded Employment columns:")
    print(employment_encoded.columns.tolist())
    print("\nFirst few rows of encoded Employment:")
    print(employment_encoded.head())
else:
    print("Employment column not found. Available columns:")
    print(df.columns.tolist())

Original Employment values:
Employment
Employed, full-time                                                                                                                                   39041
Independent contractor, freelancer, or self-employed                                                                                                   4846
Student, full-time                                                                                                                                     4709
Employed, full-time;Independent contractor, freelancer, or self-employed                                                                               3557
Not employed, but looking for work                                                                                                                     2341
                                                                                                                                                      ...  
Not employed, but looking

### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [9]:
# Identify columns with highest missing values
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_percentage = (missing_values / len(df) * 100).round(2)

missing_summary = pd.DataFrame({
    'Missing_Count': missing_values,
    'Missing_Percentage': missing_percentage
})

print("Top 15 columns with most missing values:")
print(missing_summary[missing_summary['Missing_Count'] > 0].head(15))

Top 15 columns with most missing values:
                               Missing_Count  Missing_Percentage
AINextMuch less integrated             64289               98.25
AINextLess integrated                  63082               96.40
AINextNo change                        52939               80.90
AINextMuch more integrated             51999               79.46
EmbeddedAdmired                        48704               74.43
EmbeddedWantToWorkWith                 47837               73.10
EmbeddedHaveWorkedWith                 43223               66.05
ConvertedCompYearly                    42002               64.19
AIToolNot interested in Using          41023               62.69
AINextMore integrated                  41009               62.67
Knowledge_9                            37802               57.77
Frequency_3                            37727               57.65
Knowledge_8                            37679               57.58
ProfessionalTech                       37673     

<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [10]:
# Impute missing values in ConvertedCompYearly with median
if 'ConvertedCompYearly' in df.columns:
    print(f"Missing values in ConvertedCompYearly before imputation: {df['ConvertedCompYearly'].isnull().sum()}")

    # Calculate median
    median_salary = df['ConvertedCompYearly'].median()
    print(f"Median salary: {median_salary}")

    # Impute missing values
    df['ConvertedCompYearly'].fillna(median_salary, inplace=True)

    print(f"Missing values in ConvertedCompYearly after imputation: {df['ConvertedCompYearly'].isnull().sum()}")
else:
    print("ConvertedCompYearly column not found.")

# Impute other numerical columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
print(f"\nImputing missing values in numerical columns: {list(numerical_cols)}")

for col in numerical_cols:
    if df[col].isnull().sum() > 0:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)
        print(f"Imputed {col} with median: {median_val}")

Missing values in ConvertedCompYearly before imputation: 42002
Median salary: 65000.0
Missing values in ConvertedCompYearly after imputation: 0

Imputing missing values in numerical columns: ['ResponseId', 'CompTotal', 'WorkExp', 'JobSatPoints_1', 'JobSatPoints_4', 'JobSatPoints_5', 'JobSatPoints_6', 'JobSatPoints_7', 'JobSatPoints_8', 'JobSatPoints_9', 'JobSatPoints_10', 'JobSatPoints_11', 'ConvertedCompYearly', 'JobSat']
Imputed CompTotal with median: 110000.0
Imputed WorkExp with median: 9.0
Imputed JobSatPoints_1 with median: 10.0
Imputed JobSatPoints_4 with median: 0.0
Imputed JobSatPoints_5 with median: 0.0
Imputed JobSatPoints_6 with median: 20.0
Imputed JobSatPoints_7 with median: 15.0
Imputed JobSatPoints_8 with median: 10.0
Imputed JobSatPoints_9 with median: 5.0
Imputed JobSatPoints_10 with median: 0.0
Imputed JobSatPoints_11 with median: 0.0
Imputed JobSat with median: 7.0


/tmp/ipython-input-1641794773.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ConvertedCompYearly'].fillna(median_salary, inplace=True)
/tmp/ipython-input-1641794773.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [11]:
# Impute missing values in RemoteWork with most frequent value
if 'RemoteWork' in df.columns:
    print(f"Missing values in RemoteWork before imputation: {df['RemoteWork'].isnull().sum()}")

    # Find most frequent value
    mode_remote = df['RemoteWork'].mode()[0]
    print(f"Most frequent RemoteWork value: {mode_remote}")

    # Impute missing values
    df['RemoteWork'].fillna(mode_remote, inplace=True)

    print(f"Missing values in RemoteWork after imputation: {df['RemoteWork'].isnull().sum()}")
else:
    print("RemoteWork column not found.")

# Impute other categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
print(f"\nImputing missing values in categorical columns:")

for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        mode_val = df[col].mode()
        if len(mode_val) > 0:
            df[col].fillna(mode_val[0], inplace=True)
            print(f"Imputed {col} with mode: {mode_val[0]}")

Missing values in RemoteWork before imputation: 10631
Most frequent RemoteWork value: Hybrid (some remote, some in-person)
Missing values in RemoteWork after imputation: 0


/tmp/ipython-input-4055265448.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['RemoteWork'].fillna(mode_remote, inplace=True)



Imputing missing values in categorical columns:
Imputed CodingActivities with mode: Hobby


/tmp/ipython-input-4055265448.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode_val[0], inplace=True)


Imputed EdLevel with mode: Bachelor’S Degree (B.A., B.S., B.Eng., Etc.)
Imputed LearnCode with mode: Other online resources (e.g., videos, blogs, forum, online community)
Imputed LearnCodeOnline with mode: Technical documentation;Blogs;Written Tutorials;Stack Overflow
Imputed TechDoc with mode: API document(s) and/or SDK document(s);User guides or README files found in the source repository;Traditional public search engine
Imputed YearsCode with mode: 10
Imputed YearsCodePro with mode: 2
Imputed DevType with mode: Developer, full-stack
Imputed OrgSize with mode: 20 to 99 employees
Imputed PurchaseInfluence with mode: I have some influence
Imputed BuyNewTool with mode: Start a free trial;Ask developers I know/work with;Visit developer communities like Stack Overflow
Imputed BuildvsBuy with mode: Is ready-to-go but also customizable for growth and targeted use cases
Imputed TechEndorse with mode: APIs;Customization;Reputation for quality and excellence
Imputed Country with mode: United S

### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [12]:
# Apply Min-Max scaling to ConvertedCompYearly
from sklearn.preprocessing import MinMaxScaler

if 'ConvertedCompYearly' in df.columns:
    # Initialize scaler
    scaler = MinMaxScaler()

    # Apply scaling
    df['ConvertedCompYearly_scaled'] = scaler.fit_transform(df[['ConvertedCompYearly']])

    print("Original ConvertedCompYearly statistics:")
    print(df['ConvertedCompYearly'].describe())
    print("\nScaled ConvertedCompYearly statistics:")
    print(df['ConvertedCompYearly_scaled'].describe())

    print("\nMin-Max scaling applied successfully!")
else:
    print("ConvertedCompYearly column not found.")

Original ConvertedCompYearly statistics:
count    6.543700e+04
mean     7.257636e+04
std      1.122207e+05
min      1.000000e+00
25%      6.500000e+04
50%      6.500000e+04
75%      6.500000e+04
max      1.625660e+07
Name: ConvertedCompYearly, dtype: float64

Scaled ConvertedCompYearly statistics:
count    65437.000000
mean         0.004464
std          0.006903
min          0.000000
25%          0.003998
50%          0.003998
75%          0.003998
max          1.000000
Name: ConvertedCompYearly_scaled, dtype: float64

Min-Max scaling applied successfully!


<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [13]:
# Log-transform ConvertedCompYearly to reduce skewness
import numpy as np

if 'ConvertedCompYearly' in df.columns:
    # Check original skewness
    original_skewness = df['ConvertedCompYearly'].skew()
    print(f"Original skewness: {original_skewness:.3f}")

    # Apply log transformation (add 1 to handle zeros)
    df['ConvertedCompYearly_log'] = np.log1p(df['ConvertedCompYearly'])

    # Check new skewness
    log_skewness = df['ConvertedCompYearly_log'].skew()
    print(f"Log-transformed skewness: {log_skewness:.3f}")

    print("\nOriginal ConvertedCompYearly statistics:")
    print(df['ConvertedCompYearly'].describe())
    print("\nLog-transformed ConvertedCompYearly statistics:")
    print(df['ConvertedCompYearly_log'].describe())

    print("\nLog transformation applied successfully!")
else:
    print("ConvertedCompYearly column not found.")

Original skewness: 87.708
Log-transformed skewness: -4.282

Original ConvertedCompYearly statistics:
count    6.543700e+04
mean     7.257636e+04
std      1.122207e+05
min      1.000000e+00
25%      6.500000e+04
50%      6.500000e+04
75%      6.500000e+04
max      1.625660e+07
Name: ConvertedCompYearly, dtype: float64

Log-transformed ConvertedCompYearly statistics:
count    65437.000000
mean        10.976053
std          0.851456
min          0.693147
25%         11.082158
50%         11.082158
75%         11.082158
max         16.604010
Name: ConvertedCompYearly_log, dtype: float64

Log transformation applied successfully!


### 7. Feature Engineering


<h5>7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:</h5>


In [14]:
# Create ExperienceLevel based on YearsCodePro
if 'YearsCodePro' in df.columns:
    print("Original YearsCodePro distribution:")
    print(df['YearsCodePro'].value_counts().sort_index())

    # Define experience level categories
    def categorize_experience(years):
        if pd.isna(years):
            return 'Unknown'
        elif years == 0:
            return 'Entry Level'
        elif 1 <= years <= 2:
            return 'Junior'
        elif 3 <= years <= 5:
            return 'Mid Level'
        elif 6 <= years <= 10:
            return 'Senior'
        else:
            return 'Expert'

    # Apply categorization
    df['ExperienceLevel'] = df['YearsCodePro'].apply(categorize_experience)

    print("\nExperienceLevel distribution:")
    print(df['ExperienceLevel'].value_counts())

    print("\nFeature engineering completed successfully!")
else:
    print("YearsCodePro column not found. Available columns:")
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    print([col for col in numerical_cols if 'year' in col.lower() or 'experience' in col.lower()])

Original YearsCodePro distribution:
YearsCodePro
1                      2639
10                     3251
11                     1312
12                     1777
13                     1127
14                     1082
15                     1635
16                      946
17                      814
18                      867
19                      516
2                     17995
20                     1549
21                      380
22                      492
23                      448
24                      632
25                      998
26                      426
27                      380
28                      342
29                      196
3                      4093
30                      689
31                      106
32                      194
33                      132
34                      169
35                      285
36                      119
37                      104
38                      134
39                       54
4                      3215

TypeError: '<=' not supported between instances of 'int' and 'str'

### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
